In [71]:
import pandas as pd
import numpy as np
import datetime

from pytz import country_timezones
import pytz

import glob

import airportsdata

import geopandas as gpd

from shapely.ops import nearest_points

In [4]:
from tqdm import tqdm
tqdm.pandas()

## Combine chunks

In [50]:
chunk_files = glob.glob("data/chunks/*")
traject_files = [file for file in chunk_files if "traject_all_rerun" in file]
characteristics_files = [file for file in chunk_files if "characteristics_all_rerun" in file]
print(len(characteristics_files))

300


In [51]:
ch_df_array = []
for file in characteristics_files: 
    df = pd.read_csv(file)
    ch_df_array.append(df) 

all_char = pd.concat(ch_df_array)
all_char

,flight_id,max_fuel,first_cruise,first_cruise_alt
0,254942095,5786.875688,2022-09-01 17:29:30+00:00,37000.0
1,254942148,3516.894652,2022-09-01 17:47:30+00:00,34000.0
2,254942098,1848.781798,2022-09-01 17:27:00+00:00,30000.0
3,254942161,4449.149948,2022-09-01 17:33:00+00:00,37000.0
4,254942068,9767.098354,2022-09-01 17:50:30+00:00,35000.0
...,...,...,...,...
1752,257010801,3943.376975,2022-11-14 19:34:30+00:00,35000.0
1753,257010770,2255.845093,2022-11-14 19:24:00+00:00,36000.0
1754,257010761,2284.443142,NaN,NaN
1755,257010948,4218.414258,2022-11-14 19:49:30+00:00,35000.0


In [52]:
all_char.to_csv("data/trajectory_characteristics.csv", index=False)

## Load and clean up data

In [6]:
data = pd.read_csv("data/datasets_with_regression.csv")
data.head()

,flight_id,date,callsign,adep,name_adep,country_code_adep,ades,name_ades,country_code_ades,actual_offblock_time,...,flown_distance,tow,dataset,great_circle_miles,mtow_fill,oew_fill,total_fuel,replacer,total_fuel_regression,total_fuel_fill
0,248763780,2022-01-01,3840d84f25d3f5fcc0a1be3076bb4039,EGLL,London Heathrow,GB,EICK,Cork,IE,2022-01-01T13:46:00Z,...,321,54748.000000,challenge,345.260072,78000.0,42600.0,2057.0,A320,2079.383478,2057.000000
1,248760618,2022-01-01,f6f610e73002b8892a239a81321f7f1d,LEBL,Barcelona,ES,KMIA,Miami,US,2022-01-01T09:55:00Z,...,4193,185441.000000,challenge,4688.045888,297000.0,138000.0,NaN,A333,70057.306105,70057.306105
2,248753824,2022-01-01,139670936660762c230ca92556ba842b,ESSA,Stockholm Arlanda,SE,KORD,Chicago O'Hare,US,2022-01-01T09:39:00Z,...,3770,230396.000000,challenge,4260.121521,242000.0,122780.0,NaN,A333,63802.357345,63802.357345
3,248753852,2022-01-01,509dc61bb54fbab0e5406067c95603e2,LSZH,Zurich,CH,KPHL,Philadelphia,US,2022-01-01T11:04:00Z,...,3607,157615.000000,challenge,4013.989694,228000.0,119000.0,NaN,A332,58742.869255,58742.869255
4,248755934,2022-01-01,d0610d000dcf26b1d7bba8103ecc393d,EIDW,Dublin,IE,EGLL,London Heathrow,GB,2022-01-01T12:36:00Z,...,305,70318.447226,challenge,279.504141,97000.0,50000.0,NaN,A321,2306.200351,2306.200351


In [7]:
characteristics = pd.read_csv("data/trajectory_characteristics.csv", on_bad_lines='skip')
print(len(characteristics))
characteristics = characteristics[characteristics["flight_id"]!="flight_id"]
print(len(characteristics))
characteristics.head()

510865
510865


,flight_id,max_fuel,first_cruise,first_cruise_alt
0,254942095,5786.875688,2022-09-01 17:29:30+00:00,37000.0
1,254942148,3516.894652,2022-09-01 17:47:30+00:00,34000.0
2,254942098,1848.781798,2022-09-01 17:27:00+00:00,30000.0
3,254942161,4449.149948,2022-09-01 17:33:00+00:00,37000.0
4,254942068,9767.098354,2022-09-01 17:50:30+00:00,35000.0


In [8]:
data["takeoff_time"] = pd.to_datetime(data["actual_offblock_time"]) + pd.to_timedelta(data['taxiout_time'], unit='min')
data.iloc[0]

flight_id                                       248763780
date                                           2022-01-01
callsign                 3840d84f25d3f5fcc0a1be3076bb4039
adep                                                 EGLL
name_adep                                 London Heathrow
country_code_adep                                      GB
ades                                                 EICK
name_ades                                            Cork
country_code_ades                                      IE
actual_offblock_time                 2022-01-01T13:46:00Z
arrival_time                         2022-01-01T15:04:56Z
aircraft_type                                        A320
wtc                                                     M
airline                  a73f82288988b79be490c6322f4c32ed
flight_duration                                        61
taxiout_time                                           18
flown_distance                                        321
tow           

In [9]:
data["day_of_week"] = pd.to_datetime(data["date"]).dt.dayofweek
data["month"] = pd.to_datetime(data["date"]).dt.month


In [10]:
plfs = pd.read_csv("data/passenger_load_factors.csv")
plfs= plfs[["month", "lf"]]
plfs

,month,lf
0,1,68.2
1,2,72.1
2,3,73.9
3,4,79.5
4,5,80.7
5,6,86.0
6,7,87.0
7,8,86.2
8,9,84.7
9,10,84.8


In [11]:
data = data.merge(plfs, on="month", how="left")

In [12]:
def get_tz(row): 
    try:
        zone = country_timezones(row["country_code_adep"])[0]
        hour = pd.to_datetime(row["actual_offblock_time"]).to_pydatetime().astimezone(pytz.timezone(zone)).time().hour
        return hour
        #print(hour)
    except: 
        return np.nan

In [13]:
data["hour_in_local"] = data.progress_apply(lambda x: get_tz(x), axis=1)
data

100%|█████████████████████████████████████████████████████████████████████████████████| 527162/527162 [08:08<00:00, 1078.90it/s]


,flight_id,date,callsign,adep,name_adep,country_code_adep,ades,name_ades,country_code_ades,actual_offblock_time,...,oew_fill,total_fuel,replacer,total_fuel_regression,total_fuel_fill,takeoff_time,day_of_week,month,lf,hour_in_local
0,248763780,2022-01-01,3840d84f25d3f5fcc0a1be3076bb4039,EGLL,London Heathrow,GB,EICK,Cork,IE,2022-01-01T13:46:00Z,...,42600.0,2057.0,A320,2079.383478,2057.000000,2022-01-01 14:04:00+00:00,5,1,68.2,13.0
1,248760618,2022-01-01,f6f610e73002b8892a239a81321f7f1d,LEBL,Barcelona,ES,KMIA,Miami,US,2022-01-01T09:55:00Z,...,138000.0,NaN,A333,70057.306105,70057.306105,2022-01-01 10:08:00+00:00,5,1,68.2,10.0
2,248753824,2022-01-01,139670936660762c230ca92556ba842b,ESSA,Stockholm Arlanda,SE,KORD,Chicago O'Hare,US,2022-01-01T09:39:00Z,...,122780.0,NaN,A333,63802.357345,63802.357345,2022-01-01 09:54:00+00:00,5,1,68.2,10.0
3,248753852,2022-01-01,509dc61bb54fbab0e5406067c95603e2,LSZH,Zurich,CH,KPHL,Philadelphia,US,2022-01-01T11:04:00Z,...,119000.0,NaN,A332,58742.869255,58742.869255,2022-01-01 11:15:00+00:00,5,1,68.2,12.0
4,248755934,2022-01-01,d0610d000dcf26b1d7bba8103ecc393d,EIDW,Dublin,IE,EGLL,London Heathrow,GB,2022-01-01T12:36:00Z,...,50000.0,NaN,A321,2306.200351,2306.200351,2022-01-01 12:50:00+00:00,5,1,68.2,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527157,258068876,2022-12-31,c9fca302ca2e28acab0eb0bb1b46f11b,LTFM,iGA Istanbul,TR,LSZH,Zurich,CH,2022-12-31T09:25:00Z,...,48500.0,7685.0,A321,7584.486943,7685.000000,2022-12-31 09:50:00+00:00,5,12,83.6,12.0
527158,258064675,2022-12-31,00f96ad0e382476649574ba044c764fc,EHAM,Amsterdam,NL,EDDF,Frankfurt,DE,2022-12-31T10:04:21Z,...,42600.0,1501.0,A320,1521.043383,1501.000000,2022-12-31 10:13:21+00:00,5,12,83.6,11.0
527159,258065436,2022-12-31,87c552b7f6d9bbd16a66e95df761c7f2,LEBL,Barcelona,ES,KJFK,New York JFK,US,2022-12-31T09:34:00Z,...,138000.0,NaN,A333,57391.988977,57391.988977,2022-12-31 09:48:00+00:00,5,12,83.6,10.0
527160,258058138,2022-12-31,2cd57e434494606c965bac87c024bda2,LIPE,Bologna,IT,LOWW,Vienna,AT,2022-12-31T09:37:00Z,...,28583.0,NaN,E190,1477.520039,1477.520039,2022-12-31 09:52:00+00:00,5,12,83.6,10.0


In [14]:
data_trim = data[["flight_id",
                "month",
                "day_of_week",
                "hour_in_local",
                "adep",
                "ades",
                "aircraft_type",
                "replacer",
               "wtc", 
               "airline", 
               "flight_duration",
               "great_circle_miles", 
               "lf", 
               "mtow_fill", 
               "oew_fill", 
               "total_fuel_fill", 
               "tow", 
                "takeoff_time", 
                 "dataset"]]

#"traject_est", 

In [15]:
data_trim

,flight_id,month,day_of_week,hour_in_local,adep,ades,aircraft_type,replacer,wtc,airline,flight_duration,great_circle_miles,lf,mtow_fill,oew_fill,total_fuel_fill,tow,takeoff_time,dataset
0,248763780,1,5,13.0,EGLL,EICK,A320,A320,M,a73f82288988b79be490c6322f4c32ed,61,345.260072,68.2,78000.0,42600.0,2057.000000,54748.000000,2022-01-01 14:04:00+00:00,challenge
1,248760618,1,5,10.0,LEBL,KMIA,B772,A333,H,5543e4dc327359ffaf5b9c0e6faaf0e1,570,4688.045888,68.2,297000.0,138000.0,70057.306105,185441.000000,2022-01-01 10:08:00+00:00,challenge
2,248753824,1,5,10.0,ESSA,KORD,A333,A333,H,8be5c854fd664bcb97fb543339f74770,554,4260.121521,68.2,242000.0,122780.0,63802.357345,230396.000000,2022-01-01 09:54:00+00:00,challenge
3,248753852,1,5,12.0,LSZH,KPHL,B788,A332,H,5543e4dc327359ffaf5b9c0e6faaf0e1,497,4013.989694,68.2,228000.0,119000.0,58742.869255,157615.000000,2022-01-01 11:15:00+00:00,challenge
4,248755934,1,5,12.0,EIDW,EGLL,A21N,A321,M,a73f82288988b79be490c6322f4c32ed,55,279.504141,68.2,97000.0,50000.0,2306.200351,70318.447226,2022-01-01 12:50:00+00:00,challenge
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527157,258068876,12,5,12.0,LTFM,LSZH,A321,A321,M,6351ec1b849adacc0cbb3b1313d8d39b,154,1080.659253,83.6,93500.0,48500.0,7685.000000,NaN,2022-12-31 09:50:00+00:00,submission
527158,258064675,12,5,11.0,EHAM,EDDF,A320,A320,M,f502877cab405652cf0dd70c2213e730,42,228.208435,83.6,78000.0,42600.0,1501.000000,NaN,2022-12-31 10:13:21+00:00,submission
527159,258065436,12,5,10.0,LEBL,KJFK,B772,A333,H,5543e4dc327359ffaf5b9c0e6faaf0e1,483,3821.564319,83.6,297000.0,138000.0,57391.988977,NaN,2022-12-31 09:48:00+00:00,submission
527160,258058138,12,5,10.0,LIPE,LOWW,E195,E190,M,5d407cb11cc29578cc3e292e743f5393,55,352.696423,83.6,50790.0,28583.0,1477.520039,NaN,2022-12-31 09:52:00+00:00,submission


In [16]:
data_trim["flight_id"] = data_trim["flight_id"].astype(str)
characteristics["flight_id"] = characteristics["flight_id"].astype(str)
data_trim = data_trim.merge(characteristics, on="flight_id", how="left")

data_trim

/tmp/ipykernel_821677/3575155925.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_trim["flight_id"] = data_trim["flight_id"].astype(str)


,flight_id,month,day_of_week,hour_in_local,adep,ades,aircraft_type,replacer,wtc,airline,...,lf,mtow_fill,oew_fill,total_fuel_fill,tow,takeoff_time,dataset,max_fuel,first_cruise,first_cruise_alt
0,248763780,1,5,13.0,EGLL,EICK,A320,A320,M,a73f82288988b79be490c6322f4c32ed,...,68.2,78000.0,42600.0,2057.000000,54748.000000,2022-01-01 14:04:00+00:00,challenge,2902.983545,NaN,NaN
1,248760618,1,5,10.0,LEBL,KMIA,B772,A333,H,5543e4dc327359ffaf5b9c0e6faaf0e1,...,68.2,297000.0,138000.0,70057.306105,185441.000000,2022-01-01 10:08:00+00:00,challenge,12681.820334,2022-01-01 10:30:00+00:00,33000.0
2,248753824,1,5,10.0,ESSA,KORD,A333,A333,H,8be5c854fd664bcb97fb543339f74770,...,68.2,242000.0,122780.0,63802.357345,230396.000000,2022-01-01 09:54:00+00:00,challenge,16490.600047,2022-01-01 10:25:30+00:00,36000.0
3,248753852,1,5,12.0,LSZH,KPHL,B788,A332,H,5543e4dc327359ffaf5b9c0e6faaf0e1,...,68.2,228000.0,119000.0,58742.869255,157615.000000,2022-01-01 11:15:00+00:00,challenge,18673.910205,2022-01-01 11:38:30+00:00,38000.0
4,248755934,1,5,12.0,EIDW,EGLL,A21N,A321,M,a73f82288988b79be490c6322f4c32ed,...,68.2,97000.0,50000.0,2306.200351,70318.447226,2022-01-01 12:50:00+00:00,challenge,3319.719170,2022-01-01 13:06:30+00:00,33000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527157,258068876,12,5,12.0,LTFM,LSZH,A321,A321,M,6351ec1b849adacc0cbb3b1313d8d39b,...,83.6,93500.0,48500.0,7685.000000,NaN,2022-12-31 09:50:00+00:00,submission,9500.597411,2022-12-31 10:14:30+00:00,36000.0
527158,258064675,12,5,11.0,EHAM,EDDF,A320,A320,M,f502877cab405652cf0dd70c2213e730,...,83.6,78000.0,42600.0,1501.000000,NaN,2022-12-31 10:13:21+00:00,submission,3070.692671,2022-12-31 10:29:30+00:00,27000.0
527159,258065436,12,5,10.0,LEBL,KJFK,B772,A333,H,5543e4dc327359ffaf5b9c0e6faaf0e1,...,83.6,297000.0,138000.0,57391.988977,NaN,2022-12-31 09:48:00+00:00,submission,13990.222635,2022-12-31 10:10:30+00:00,33000.0
527160,258058138,12,5,10.0,LIPE,LOWW,E195,E190,M,5d407cb11cc29578cc3e292e743f5393,...,83.6,50790.0,28583.0,1477.520039,NaN,2022-12-31 09:52:00+00:00,submission,1791.295075,2022-12-31 10:12:30+00:00,32000.0


In [17]:
data_trim["time_to_cruise"] = (pd.to_datetime(data_trim["first_cruise"]) - pd.to_datetime(data_trim["takeoff_time"])).dt.seconds
data_trim["alt_per_s"] = data_trim["first_cruise_alt"].astype(float) / data_trim["time_to_cruise"].astype(float)
data_trim = data_trim.drop(columns=["first_cruise", "takeoff_time"])

data_trim["est_load_lf_adjusted"] = (data_trim["mtow_fill"] -  data_trim["oew_fill"] - data_trim["total_fuel_fill"]) * data_trim["lf"] / 100
data_trim["est_tow"] = data_trim["est_load_lf_adjusted"] + data_trim["oew_fill"] + data_trim["total_fuel_fill"]

data_trim

,flight_id,month,day_of_week,hour_in_local,adep,ades,aircraft_type,replacer,wtc,airline,...,oew_fill,total_fuel_fill,tow,dataset,max_fuel,first_cruise_alt,time_to_cruise,alt_per_s,est_load_lf_adjusted,est_tow
0,248763780,1,5,13.0,EGLL,EICK,A320,A320,M,a73f82288988b79be490c6322f4c32ed,...,42600.0,2057.000000,54748.000000,challenge,2902.983545,NaN,NaN,NaN,22739.926000,67396.926000
1,248760618,1,5,10.0,LEBL,KMIA,B772,A333,H,5543e4dc327359ffaf5b9c0e6faaf0e1,...,138000.0,70057.306105,185441.000000,challenge,12681.820334,33000.0,1320.0,25.000000,60658.917236,268716.223341
2,248753824,1,5,10.0,ESSA,KORD,A333,A333,H,8be5c854fd664bcb97fb543339f74770,...,122780.0,63802.357345,230396.000000,challenge,16490.600047,36000.0,1890.0,19.047619,37794.832291,224377.189636
3,248753852,1,5,12.0,LSZH,KPHL,B788,A332,H,5543e4dc327359ffaf5b9c0e6faaf0e1,...,119000.0,58742.869255,157615.000000,challenge,18673.910205,38000.0,1410.0,26.950355,34275.363168,212018.232423
4,248755934,1,5,12.0,EIDW,EGLL,A21N,A321,M,a73f82288988b79be490c6322f4c32ed,...,50000.0,2306.200351,70318.447226,challenge,3319.719170,33000.0,990.0,33.333333,30481.171361,82787.371712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527157,258068876,12,5,12.0,LTFM,LSZH,A321,A321,M,6351ec1b849adacc0cbb3b1313d8d39b,...,48500.0,7685.000000,NaN,submission,9500.597411,36000.0,1470.0,24.489796,31195.340000,87380.340000
527158,258064675,12,5,11.0,EHAM,EDDF,A320,A320,M,f502877cab405652cf0dd70c2213e730,...,42600.0,1501.000000,NaN,submission,3070.692671,27000.0,969.0,27.863777,28339.564000,72440.564000
527159,258065436,12,5,10.0,LEBL,KJFK,B772,A333,H,5543e4dc327359ffaf5b9c0e6faaf0e1,...,138000.0,57391.988977,NaN,submission,13990.222635,33000.0,1350.0,24.444444,84944.297215,280336.286192
527160,258058138,12,5,10.0,LIPE,LOWW,E195,E190,M,5d407cb11cc29578cc3e292e743f5393,...,28583.0,1477.520039,NaN,submission,1791.295075,32000.0,1230.0,26.016260,17329.845248,47390.365286


## Variable types and rounding

In [18]:
data_trim.iloc[0]

flight_id                                      248763780
month                                                  1
day_of_week                                            5
hour_in_local                                       13.0
adep                                                EGLL
ades                                                EICK
aircraft_type                                       A320
replacer                                            A320
wtc                                                    M
airline                 a73f82288988b79be490c6322f4c32ed
flight_duration                                       61
great_circle_miles                            345.260072
lf                                                  68.2
mtow_fill                                        78000.0
oew_fill                                         42600.0
total_fuel_fill                                   2057.0
tow                                              54748.0
dataset                        

In [19]:
data_trim["flight_id"] = data_trim["flight_id"].astype(str)
print(len(data_trim[data_trim["flight_id"].isna()]) / len(data_trim["flight_id"]))

0.0


In [20]:
data_trim["month"] = data_trim["month"].astype(int)
print(len(data_trim[data_trim["month"].isna()]) / len(data_trim["month"]))

0.0


In [21]:
data_trim["day_of_week"] = data_trim["day_of_week"].astype(int)
print(len(data_trim[data_trim["day_of_week"].isna()]) / len(data_trim["day_of_week"]))

0.0


In [22]:
data_trim["adep"] = data_trim["adep"].astype(str)
print(len(data_trim[data_trim["adep"].isna()]) / len(data_trim["adep"]))

0.0


In [23]:
data_trim["ades"] = data_trim["ades"].astype(str)
print(len(data_trim[data_trim["ades"].isna()]) / len(data_trim["ades"]))

0.0


In [24]:
data_trim["aircraft_type"] = data_trim["aircraft_type"].astype(str)
print(len(data_trim[data_trim["aircraft_type"].isna()]) / len(data_trim["aircraft_type"]))

0.0


In [25]:
data_trim = data_trim.drop(columns=["wtc"])

In [26]:
data_trim["airline"] = data_trim["airline"].astype(str)
print(len(data_trim[data_trim["airline"].isna()]) / len(data_trim["airline"]))

0.0


In [27]:
data_trim["flight_duration_sec"] = data_trim["flight_duration"].astype(int) * 60
data_trim = data_trim.drop(columns=["flight_duration"])
print(len(data_trim[data_trim["flight_duration_sec"].isna()]) / len(data_trim["flight_duration_sec"]))

0.0


In [28]:
data_trim["great_circle_km"] = data_trim["great_circle_miles"] * 1.60934
data_trim["great_circle_km"] = data_trim["great_circle_km"].astype(int)
data_trim = data_trim.drop(columns=["great_circle_miles"])
print(len(data_trim[data_trim["great_circle_km"].isna()]) / len(data_trim["great_circle_km"]))

0.0


In [29]:
data_trim = data_trim.drop(columns=["lf"])

In [30]:
data_trim["mtow_fill"] = data_trim["mtow_fill"].apply(lambda x: round(x, 0)).astype(int)
print(len(data_trim[data_trim["mtow_fill"].isna()]) / len(data_trim["mtow_fill"]))

0.0


In [31]:
data_trim["oew_fill"] = data_trim["oew_fill"].apply(lambda x: round(x, 0)).astype(int)
print(len(data_trim[data_trim["oew_fill"].isna()]) / len(data_trim["oew_fill"]))

0.0


In [32]:
data_trim["total_fuel_fill"] = data_trim["total_fuel_fill"].apply(lambda x: round(x, 0)).astype(int)
print(len(data_trim[data_trim["total_fuel_fill"].isna()]) / len(data_trim["total_fuel_fill"]))

0.0


In [33]:
data_trim["tow"] = data_trim["tow"].apply(lambda x: round(x, 0)).astype(float)
print(len(data_trim[data_trim["tow"].isna()]) / len(data_trim["tow"]))

0.30000075878003346


In [34]:
data_trim = data_trim.drop(columns=["max_fuel"])

In [35]:
data_trim["first_cruise_alt"] = data_trim["first_cruise_alt"].astype(float).apply(lambda x: round(x, 0))
print(1 - (len(data_trim[data_trim["first_cruise_alt"].isna()]) / len(data_trim["first_cruise_alt"])))



0.8445449406444319


In [36]:
data_trim["time_to_cruise"] = data_trim["time_to_cruise"].astype(float).apply(lambda x: round(x, 0))
print(1 - (len(data_trim[data_trim["time_to_cruise"].isna()]) / len(data_trim["time_to_cruise"])))


0.8445449406444319


In [37]:
data_trim["alt_per_s"] = data_trim["alt_per_s"].astype(float).apply(lambda x: round(x, 1))
print(1 - (len(data_trim[data_trim["alt_per_s"].isna()]) / len(data_trim["alt_per_s"])))


0.8445449406444319


In [38]:
data_trim["est_load_lf_adjusted"] = data_trim["est_load_lf_adjusted"].astype(float).apply(lambda x: round(x, 0))
print(1 - (len(data_trim[data_trim["est_load_lf_adjusted"].isna()]) / len(data_trim["est_load_lf_adjusted"])))


1.0


In [39]:
data_trim["est_tow"] = data_trim["est_tow"].astype(float).apply(lambda x: round(x, 0))
print(1 - (len(data_trim[data_trim["est_tow"].isna()]) / len(data_trim["est_tow"])))


1.0


In [40]:
data_trim

,flight_id,month,day_of_week,hour_in_local,adep,ades,aircraft_type,replacer,airline,mtow_fill,...,total_fuel_fill,tow,dataset,first_cruise_alt,time_to_cruise,alt_per_s,est_load_lf_adjusted,est_tow,flight_duration_sec,great_circle_km
0,248763780,1,5,13.0,EGLL,EICK,A320,A320,a73f82288988b79be490c6322f4c32ed,78000,...,2057,54748.0,challenge,NaN,NaN,NaN,22740.0,67397.0,3660,555
1,248760618,1,5,10.0,LEBL,KMIA,B772,A333,5543e4dc327359ffaf5b9c0e6faaf0e1,297000,...,70057,185441.0,challenge,33000.0,1320.0,25.0,60659.0,268716.0,34200,7544
2,248753824,1,5,10.0,ESSA,KORD,A333,A333,8be5c854fd664bcb97fb543339f74770,242000,...,63802,230396.0,challenge,36000.0,1890.0,19.0,37795.0,224377.0,33240,6855
3,248753852,1,5,12.0,LSZH,KPHL,B788,A332,5543e4dc327359ffaf5b9c0e6faaf0e1,228000,...,58743,157615.0,challenge,38000.0,1410.0,27.0,34275.0,212018.0,29820,6459
4,248755934,1,5,12.0,EIDW,EGLL,A21N,A321,a73f82288988b79be490c6322f4c32ed,97000,...,2306,70318.0,challenge,33000.0,990.0,33.3,30481.0,82787.0,3300,449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527157,258068876,12,5,12.0,LTFM,LSZH,A321,A321,6351ec1b849adacc0cbb3b1313d8d39b,93500,...,7685,NaN,submission,36000.0,1470.0,24.5,31195.0,87380.0,9240,1739
527158,258064675,12,5,11.0,EHAM,EDDF,A320,A320,f502877cab405652cf0dd70c2213e730,78000,...,1501,NaN,submission,27000.0,969.0,27.9,28340.0,72441.0,2520,367
527159,258065436,12,5,10.0,LEBL,KJFK,B772,A333,5543e4dc327359ffaf5b9c0e6faaf0e1,297000,...,57392,NaN,submission,33000.0,1350.0,24.4,84944.0,280336.0,28980,6150
527160,258058138,12,5,10.0,LIPE,LOWW,E195,E190,5d407cb11cc29578cc3e292e743f5393,50790,...,1478,NaN,submission,32000.0,1230.0,26.0,17330.0,47390.0,3300,567


In [89]:
data_trim[data_trim["dataset"]=="challenge"].to_csv("data/STAGE_1_TRAIN.csv", index=False)

In [90]:
data_trim.to_csv("data/STAGE_1_ALL.csv", index=False)

## Find submission adep and ades values that aren't in training data

In [140]:
data_trim = pd.read_csv("data/STAGE_1_ALL.csv")

In [144]:
stage_1_train_data = data_trim[data_trim["dataset"]=="challenge"]
stage_1_train_data

,flight_id,month,day_of_week,hour_in_local,adep,ades,aircraft_type,replacer,airline,mtow_fill,...,total_fuel_fill,tow,dataset,first_cruise_alt,time_to_cruise,alt_per_s,est_load_lf_adjusted,est_tow,flight_duration_sec,great_circle_km
0,248763780,1,5,13.0,EGLL,EICK,A320,A320,a73f82288988b79be490c6322f4c32ed,78000,...,2057,54748.0,challenge,NaN,NaN,NaN,22740.0,67397.0,3660,555
1,248760618,1,5,10.0,LEBL,KMIA,B772,A333,5543e4dc327359ffaf5b9c0e6faaf0e1,297000,...,70057,185441.0,challenge,33000.0,1320.0,25.0,60659.0,268716.0,34200,7544
2,248753824,1,5,10.0,ESSA,KORD,A333,A333,8be5c854fd664bcb97fb543339f74770,242000,...,63802,230396.0,challenge,36000.0,1890.0,19.0,37795.0,224377.0,33240,6855
3,248753852,1,5,12.0,LSZH,KPHL,B788,A332,5543e4dc327359ffaf5b9c0e6faaf0e1,228000,...,58743,157615.0,challenge,38000.0,1410.0,27.0,34275.0,212018.0,29820,6459
4,248755934,1,5,12.0,EIDW,EGLL,A21N,A321,a73f82288988b79be490c6322f4c32ed,97000,...,2306,70318.0,challenge,33000.0,990.0,33.3,30481.0,82787.0,3300,449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369008,258058361,12,5,10.0,LFPG,KMIA,B788,A332,5543e4dc327359ffaf5b9c0e6faaf0e1,228000,...,66892,163438.0,challenge,33975.0,1290.0,26.3,35202.0,221094.0,32700,7371
369009,258071247,12,5,12.0,LTFM,EDDB,A21N,A321,6351ec1b849adacc0cbb3b1313d8d39b,97000,...,7364,78707.0,challenge,34000.0,1410.0,24.1,33136.0,90500.0,9480,1685
369010,258059152,12,5,10.0,EDDL,EIDW,A320,A320,a73f82288988b79be490c6322f4c32ed,78000,...,3129,62943.0,challenge,36000.0,2610.0,13.8,26979.0,72708.0,5940,915
369011,258072276,12,5,10.0,LFPG,EIDW,A21N,A321,a73f82288988b79be490c6322f4c32ed,97000,...,3678,72611.0,challenge,36000.0,1350.0,26.7,36217.0,89895.0,5040,784


In [166]:
stage_1_train_data = pd.read_csv("data/train_actual.csv")
stage_1_train_data

,flight_id,month,day_of_week,hour_in_local,adep,ades,aircraft_type,replacer,airline,mtow_fill,...,total_fuel_fill,tow,dataset,first_cruise_alt,time_to_cruise,alt_per_s,est_load_lf_adjusted,est_tow,flight_duration_sec,great_circle_km
0,248763780,1,5,13.0,EGLL,EICK,A320,A320,a73f82288988b79be490c6322f4c32ed,78000,...,2057,54748,challenge,NaN,NaN,NaN,22740,67397,3660,555
1,248760618,1,5,10.0,LEBL,KMIA,B772,A333,5543e4dc327359ffaf5b9c0e6faaf0e1,297000,...,70057,185441,challenge,33000.0,1320.0,25.0,60659,268716,34200,7544
2,248753852,1,5,12.0,LSZH,KPHL,B788,A332,5543e4dc327359ffaf5b9c0e6faaf0e1,228000,...,58743,157615,challenge,38000.0,1410.0,27.0,34275,212018,29820,6459
3,248755934,1,5,12.0,EIDW,EGLL,A21N,A321,a73f82288988b79be490c6322f4c32ed,97000,...,2306,70318,challenge,33000.0,990.0,33.3,30481,82787,3300,449
4,248762583,1,5,11.0,ENGM,LEAL,A20N,B738,8be5c854fd664bcb97fb543339f74770,79000,...,8618,57721,challenge,38000.0,1028.0,37.0,17788,70706,12840,2570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350578,258058361,12,5,10.0,LFPG,KMIA,B788,A332,5543e4dc327359ffaf5b9c0e6faaf0e1,228000,...,66892,163438,challenge,33975.0,1290.0,26.3,35202,221094,32700,7371
350579,258071247,12,5,12.0,LTFM,EDDB,A21N,A321,6351ec1b849adacc0cbb3b1313d8d39b,97000,...,7364,78707,challenge,34000.0,1410.0,24.1,33136,90500,9480,1685
350580,258059152,12,5,10.0,EDDL,EIDW,A320,A320,a73f82288988b79be490c6322f4c32ed,78000,...,3129,62943,challenge,36000.0,2610.0,13.8,26979,72708,5940,915
350581,258072276,12,5,10.0,LFPG,EIDW,A21N,A321,a73f82288988b79be490c6322f4c32ed,97000,...,3678,72611,challenge,36000.0,1350.0,26.7,36217,89895,5040,784


In [167]:
not_sub = stage_1_train_data
sub = pd.read_csv("/mnt/SMB_share/mark/flight_competition/final_submission_set.csv")

In [168]:
challenge_set = pd.read_csv("/mnt/SMB_share/mark/flight_competition/challenge_set.csv")
challenge_set

,flight_id,date,callsign,adep,name_adep,country_code_adep,ades,name_ades,country_code_ades,actual_offblock_time,arrival_time,aircraft_type,wtc,airline,flight_duration,taxiout_time,flown_distance,tow
0,248763780,2022-01-01,3840d84f25d3f5fcc0a1be3076bb4039,EGLL,London Heathrow,GB,EICK,Cork,IE,2022-01-01T13:46:00Z,2022-01-01T15:04:56Z,A320,M,a73f82288988b79be490c6322f4c32ed,61,18,321,54748.000000
1,248760618,2022-01-01,f6f610e73002b8892a239a81321f7f1d,LEBL,Barcelona,ES,KMIA,Miami,US,2022-01-01T09:55:00Z,2022-01-01T19:37:56Z,B772,H,5543e4dc327359ffaf5b9c0e6faaf0e1,570,13,4193,185441.000000
2,248753824,2022-01-01,139670936660762c230ca92556ba842b,ESSA,Stockholm Arlanda,SE,KORD,Chicago O'Hare,US,2022-01-01T09:39:00Z,2022-01-01T19:08:13Z,A333,H,8be5c854fd664bcb97fb543339f74770,554,15,3770,230396.000000
3,248753852,2022-01-01,509dc61bb54fbab0e5406067c95603e2,LSZH,Zurich,CH,KPHL,Philadelphia,US,2022-01-01T11:04:00Z,2022-01-01T19:32:13Z,B788,H,5543e4dc327359ffaf5b9c0e6faaf0e1,497,11,3607,157615.000000
4,248755934,2022-01-01,d0610d000dcf26b1d7bba8103ecc393d,EIDW,Dublin,IE,EGLL,London Heathrow,GB,2022-01-01T12:36:00Z,2022-01-01T13:44:32Z,A21N,M,a73f82288988b79be490c6322f4c32ed,55,14,305,70318.447226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369008,258058361,2022-12-31,85ee68e5b7b5acf24ba00d1318eca1e8,LFPG,Paris Charles de Gaulle,FR,KMIA,Miami,US,2022-12-31T09:38:00Z,2022-12-31T19:03:48Z,B788,H,5543e4dc327359ffaf5b9c0e6faaf0e1,545,21,4046,163438.000000
369009,258071247,2022-12-31,570cf7d5ebbd691bcba63e7466607da7,LTFM,iGA Istanbul,TR,EDDB,Berlin Brandenburg,DE,2022-12-31T09:27:00Z,2022-12-31T12:29:41Z,A21N,M,6351ec1b849adacc0cbb3b1313d8d39b,158,25,946,78707.000000
369010,258059152,2022-12-31,5a7e43e4f981539ae3d3b1cb31591b7c,EDDL,Dusseldorf,DE,EIDW,Dublin,IE,2022-12-31T09:52:00Z,2022-12-31T11:41:51Z,A320,M,a73f82288988b79be490c6322f4c32ed,99,11,522,62942.750000
369011,258072276,2022-12-31,a1c078516f9f9e90cacec61854cad45b,LFPG,Paris Charles de Gaulle,FR,EIDW,Dublin,IE,2022-12-31T09:37:00Z,2022-12-31T11:13:10Z,A21N,M,a73f82288988b79be490c6322f4c32ed,84,12,466,72611.161024


In [169]:
missing_adep = sub[~sub["adep"].isin(not_sub["adep"].unique())]["adep"].unique()
missing_ades = sub[~sub["ades"].isin(not_sub["ades"].unique())]["ades"].unique()

In [170]:
print(len(sub[sub["adep"]=="VAJJ"]))
print(len(challenge_set[challenge_set["adep"]=="VAJJ"]))

0
0


In [190]:



# missing_adep = ["7FL6", "CYAW", "DAOL", "EDBA", "EDTB", "EG73", "EGJB", "EGVA", "ENKA", "EPKM", ...11 not listed..., "LTBW", "MMEP", "OS66", "RJAK", "RJTK", "UDYE", "UGEJ", "UKOM", "VAJJ", "WICB"]
# missing_ades = ["7FL6", "CYAW", "DAOL", "EDBA", "EDMT", "EDTB", "EG73", "EGJB", "EGVA", "EPKM", ...7 not listed..., "LTAQ", "LTCO", "OS66", "RJAK", "UDYE", "UGEJ", "UKOM", "VAJJ", "VHSK", "WICB"]
missing = np.concatenate((missing_adep, missing_ades), axis=None)
missing = list(set(missing))
missing = missing + ["HRYR"] + ["LTAW", "LTCN", "LTCO", "LTCT", "RKSI", "RPLL", "WADD"] + ["HTZA"] + ["LTCP", "LTCW", "VVTS", "WMKK", "ZMCK"] + ["VTBS"] + ["HTDA", "LTCL", "VTSG"] + ["HAAB", "HCMM", "LTCV"]

missing



['EGPE',
 'EDSB',
 'UBBN',
 'LTAP',
 'LKTB',
 'WIII',
 'EDDE',
 'RJAA',
 'VABB',
 'LTAJ',
 'LTCI',
 'LTBU',
 'LEAB',
 'RJTT',
 'UKOO',
 'KDAB',
 'LFLX',
 'KRFD',
 'VHHH',
 'OMAA',
 'EFET',
 'LIPO',
 'LGBL',
 'UDYZ',
 'EDJA',
 'EPKT',
 'HHAS',
 'DAOO',
 'LTCM',
 'CYHZ',
 'LKMT',
 'HUEN',
 'LFBH',
 'EGJJ',
 'ESKN',
 'EGVN',
 'MMPR',
 'HRYR',
 'LTAW',
 'LTCN',
 'LTCO',
 'LTCT',
 'RKSI',
 'RPLL',
 'WADD',
 'HTZA',
 'LTCP',
 'LTCW',
 'VVTS',
 'WMKK',
 'ZMCK',
 'VTBS',
 'HTDA',
 'LTCL',
 'VTSG',
 'HAAB',
 'HCMM',
 'LTCV']

In [191]:
manual_airports = {
'LTCU':{'icao': 'LTCU',
 'lat': 38.85858736260065,
 'lon': 40.59067139686379 
}, 
'LEIB':{'icao': 'LEIB',
 'lat': 38.87392171890811,
 'lon': 1.3722346680281414 
}, 
'DTNH':{'icao': 'DTNH',
 'lat': 36.0834437013865,
 'lon': 10.438704017504692 
}, 
'ESKS':{'icao': 'ESKS',
 'lat': 61.156101445931895,
 'lon': 12.837457063067493 
}, 
'EBCI':{'icao': 'EBCI',
 'lat': 50.463130022524915,
 'lon': 4.462079975151949 
}, 
'EBMB':{'icao': 'EBMB',
 'lat': 50.9154369162626,
 'lon': 4.493584452729591 
}, 
'LTCV':{'icao': 'LTCV',
 'lat': 37.36158697994315,
 'lon': 42.05553763068404
}, 
'OKBK':{'icao': 'OKBK',
 'lat': 37.36160005163285,
 'lon': 42.055506593027715 
}, 
'EGHH':{'icao': 'EGHH',
 'lat': 50.78100421147004,
 'lon': -1.8409717739922666 
}, 
'EGLL':{'icao': 'EGLL',
 'lat': 51.46794534067239,
 'lon': -0.45500243490457304 
}, 
'EKCH':{'icao': 'EKCH',
 'lat': 55.60955846679605,
 'lon': 12.63561794162437 
}, 
'LTFJ':{'icao': 'LTFJ',
 'lat': 40.90849582099669,
 'lon': 29.315195387558404 
}, 
'UTFF':{'icao': 'UTFF',
 'lat': 40.37598329652268,
 'lon': 71.75202477850446 
}, 
'EDDV':{'icao': 'EDDV',
 'lat': 52.462666312589754,
 'lon': 9.684258394471607
}, 
'EBBR':{'icao': 'EBBR',
 'lat': 50.90019751841645,
 'lon': 4.486028487284325
}, 
'EGCN':{'icao': 'EGCN',
 'lat': 53.39443776608614,
 'lon': -1.3842738109711732
}, 
'EGCC':{'icao': 'EGCC',
 'lat': 53.35534409298594,
 'lon':  -2.2771244490697478
}, 
'EGBB':{'icao': 'EGBB',
 'lat': 52.45236082325583,
 'lon': -1.7435028355783422
}, 
'EGKK':{'icao': 'EGKK',
 'lat': 51.15364864102023,
 'lon': -0.18207899325080945
}, 
'EKYT':{'icao': 'EKYT',
 'lat': 57.095703276939744,
 'lon': 9.855358741769924
}, 
'LTAC':{'icao': 'LTAC',
 'lat': 40.11496213059108,
 'lon': 32.99271320586336
}, 
'EHAM':{'icao': 'EHAM',
 'lat': 52.31690578306712,
 'lon': 4.74592996442235
}, 
'EGSY':{'icao': 'EGSY',
 'lat': 53.39926177010993,
 'lon': -1.3841440642047664
}, 
'LTFM':{'icao': 'LTFM',
 'lat': 41.275238322317286,
 'lon': 28.732290831634593
}}

In [192]:
airports = airportsdata.load()
airports.update(manual_airports)

In [193]:
for x in missing: 
    try: 
        found = airports[x]
    except: 
        print(x)

In [194]:
airport_df = pd.DataFrame([[x, airports[x]["lat"], airports[x]["lon"]] for x in airports.keys()], columns=["icao", "lat", "lng"])
airport_gdf = gpd.GeoDataFrame(airport_df, geometry=gpd.points_from_xy(airport_df["lng"], airport_df["lat"]), crs=4326)
airport_gdf

,icao,lat,lng,geometry
0,00AA,38.704020,-101.473910,POINT (-101.47391 38.70402)
1,00AK,59.948890,-151.692220,POINT (-151.69222 59.94889)
2,00AL,34.864810,-86.770280,POINT (-86.77028 34.86481)
3,00AN,59.093470,-156.455830,POINT (-156.45583 59.09347)
4,00AR,38.969650,-97.601560,POINT (-97.60156 38.96965)
...,...,...,...,...
28229,_KBH,-3.791670,32.646670,POINT (32.64667 -3.79167)
28230,_OUK,60.425580,-0.746600,POINT (-0.74660 60.42558)
28231,_YEH,38.481900,106.009000,POINT (106.00900 38.48190)
28232,OKBK,37.361600,42.055507,POINT (42.05551 37.36160)


In [195]:
missing_gpd = airport_gdf[airport_gdf["icao"].isin(missing)]
not_missing = np.concatenate((challenge_set["adep"].unique(), challenge_set["ades"].unique()), axis=None)
not_missing_gpd = airport_gdf[(airport_gdf["icao"].isin(not_missing)) & (~airport_gdf["icao"].isin(missing))]

In [196]:
pts3 = not_missing_gpd.geometry.unary_union
def near(point, pts=pts3):
     # find the nearest point and return the corresponding Place value
     nearest = not_missing_gpd.geometry == nearest_points(point, pts)[1]
     return not_missing_gpd[nearest].icao.iloc[0]
    
missing_gpd['nearest'] = missing_gpd.apply(lambda row: near(row.geometry), axis=1)
missing_gpd

/home/mark/anaconda3/envs/traffic/lib/python3.10/site-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,icao,lat,lng,geometry,nearest
6019,CYHZ,44.880800,-63.508600,POINT (-63.50860 44.88080),KBOS
6336,DAOO,35.623900,-0.621180,POINT (-0.62118 35.62390),GMFO
6661,EDDE,50.979800,10.958100,POINT (10.95810 50.97980),EDDP
6744,EDJA,47.988800,10.239500,POINT (10.23950 47.98880),EDNY
6916,EDSB,48.779400,8.080500,POINT (8.08050 48.77940),LFST
7038,EFET,68.362600,23.424300,POINT (23.42430 68.36260),EFKT
7231,EGJJ,49.207900,-2.195510,POINT (-2.19551 49.20790),EGHH
7289,EGPE,57.542500,-4.047500,POINT (-4.04750 57.54250),EGPF
7354,EGVN,51.750000,-1.583620,POINT (-1.58362 51.75000),EGBB
7621,EPKT,50.474300,19.080000,POINT (19.08000 50.47430),EPKK


In [197]:
with_replacement = data_trim.copy()
for icao in missing_gpd["icao"].unique(): 
    print(icao) 
    replacer = missing_gpd[missing_gpd["icao"]==icao]["nearest"].iloc[0]
    print(replacer)
    with_replacement["adep"] = with_replacement["adep"].replace(icao, replacer)
    with_replacement["ades"] = with_replacement["ades"].replace(icao, replacer)
    

CYHZ
KBOS
DAOO
GMFO
EDDE
EDDP
EDJA
EDNY
EDSB
LFST
EFET
EFKT
EGJJ
EGHH
EGPE
EGPF
EGVN
EGBB
EPKT
EPKK
ESKN
ESSP
HAAB
HSSK
HCMM
HSSK
HHAS
HSSK
HRYR
FZAA
HTDA
FAOR
HTZA
FAOR
HUEN
HSSK
KDAB
KMCO
KRFD
KORD
LEAB
LEMI
LFBH
LFRS
LFLX
LFPO
LGBL
LGSK
LIPO
LIPX
LKMT
EPKK
LKTB
LOWW
LTAJ
LTDA
LTAP
LTFH
LTAW
LTAR
LTBU
LTFM
LTCI
OKBK
LTCL
OKBK
LTCM
LTAL
LTCN
LTDA
LTCO
LTCF
LTCP
LTCS
LTCT
LTCF
LTCV
OKBK
LTCW
ORER
MMPR
KIAH
OMAA
OMDW
RJAA
VTSP
RJTT
VTSP
RKSI
VGHS
RPLL
VTSP
UBBN
UBBG
UDYZ
LTCF
UKOO
LUKK
VABB
VOGO
VHHH
VTSP
VTBS
VTSP
VTSG
VTSP
VVTS
VTSP
WADD
VTSP
WIII
VTSP
WMKK
VTSP
ZMCK
VGHS


In [198]:
with_replacement.to_csv("data/STAGE_1_ALL_rep.csv", index=False)